In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-dw_kkl42
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-dw_kkl42
  Resolved https://github.com/huggingface/transformers.git to commit 6690ba3f4d036bc39bdf29ec98daf2c693442503
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 597.7 kB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.22.0.dev0-py3-none-any.whl size=4801811 sha256=ff6f9aa34b8315e8d1bc5a423d602e27b2c206fd573c64d354e9bfed8995a54f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1q9zefgu/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0

In [3]:
import transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm

import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import gc
import re
tqdm.pandas()
print(torch.__version__)
transformers.__version__

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

1.11.0


'4.22.0.dev0'

In [4]:
%%time

model = AutoModelForSeq2SeqLM.from_pretrained("../input/huggingface-meta-nllb200/nllb-200-distilled-1.3B")
tokenizer = AutoTokenizer.from_pretrained("../input/huggingface-meta-nllb200/nllb-200-distilled-1.3B")

CPU times: user 17.6 s, sys: 7.93 s, total: 25.5 s
Wall time: 1min 7s


In [5]:
%%time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
translation_pipeline = pipeline('translation', 
                                model=model, 
                                tokenizer=tokenizer, 
                                src_lang="arb_Arab",  #arb_Arab,eng_latn
                                tgt_lang='eng_latn',
                                device = device,
                                max_length = 1024)

translation_pipeline_bn = pipeline('translation', 
                                model=model, 
                                tokenizer=tokenizer, 
                                src_lang="arb_Arab",  #arb_Arab,eng_latn
                                tgt_lang='ben_Beng',
                                device = device,
                                max_length = 1024)

cuda:0
CPU times: user 2.68 s, sys: 683 ms, total: 3.36 s
Wall time: 6.19 s


In [6]:
en_bn_translated_sahih_hadiths = pd.read_csv('../input/en-bn-sahih-bukhari-muslim/en_bn_translated_sahih_hadiths.csv')
en_bn_translated_sahih_hadiths.head()

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...,'Abdullah bin al-Zubair bin 'Isa عبد الله بن ا...,উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে ...
1,1,2,Sahih Bukhari,1,2,Revelation - كتاب بدء الوحى,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...,'Abdullah bin Yusuf al-Tunaysi عبد الله بن يوس...,"আয়েশা (রাঃ) এর কাহিনী বর্ণিত হয়েছে, তিনি বলে..."
2,2,3,Sahih Bukhari,1,3,Revelation - كتاب بدء الوحى,"30399, 20023, 11207, 11013, 10511, 53",حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...,Narrated 'Aisha: (the m...,Yahya bin Yahya bin Bukayr يحيى بن يحيى بن بكي...,'আয়েশা (রাঃ) এর বানী বর্ণিত হয়েছেঃ (বিশ্বাসী...
3,3,4,Sahih Bukhari,1,4,Revelation - كتاب بدء الوحى,"11013, 10567, 34",قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...,Narrated Jabir bin 'Abdullah Al-Ansari while ...,"al-Zuhri ابن شهاب الزهري‎, محمد بن مسلم,Abu Sa...",জাবির বিন 'আব্দুল্লাহ আল-আনসারি বর্ণনা করেছেন ...
4,4,5,Sahih Bukhari,1,5,Revelation - كتاب بدء الوحى,"20040, 20469, 11399, 11050, 17",حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...,Narrated Said bin Jubair: ...,Musa bin Isma'il al-Munqari al-Tabudhaki موسى ...,"তিনি বলেন, ইবনে জুবাইর বলেনঃ ইবনে আব্বাস আল্লা..."


In [7]:

df = en_bn_translated_sahih_hadiths[en_bn_translated_sahih_hadiths['text_bn'].str.contains('এখনো পাওয়া যায়নি')]
len(df)

531

In [8]:
df = en_bn_translated_sahih_hadiths[en_bn_translated_sahih_hadiths['text_ar'].str.contains('not found yet')]
len(df)

6

In [9]:
df = en_bn_translated_sahih_hadiths[en_bn_translated_sahih_hadiths['text_en'].str.contains('not found yet')]
len(df)

531

In [10]:
len(df.text_en[3630])

13

In [11]:
en_bn_translated_sahih_hadiths.text_ar[0]

'حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب  رضى الله عنه  على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول \u200f"\u200f إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه \u200f"\u200f\u200f.\u200f'

In [12]:
%%time

def ar_to_en_bn_translator(en_text,ar_text,to_bn = False):
    if(('not found yet' in en_text) or ('এখনো পাওয়া যায়নি' in en_text)):
        pass
    
    else:
        return en_text,False
#     sentences = re.split(r'(?<=\w\.)\s', str(en_text))

    if(to_bn):
        result = translation_pipeline_bn(ar_text)
    else:
        result = translation_pipeline(ar_text)
    return result[0]['translation_text'],True
  
for i in range(len(en_bn_translated_sahih_hadiths)):
    text,check_change = ar_to_en_bn_translator(en_bn_translated_sahih_hadiths.text_en[i],en_bn_translated_sahih_hadiths.text_ar[i])
    if(check_change):
        text,check_change = ar_to_en_bn_translator(en_bn_translated_sahih_hadiths.text_bn[i],en_bn_translated_sahih_hadiths.text_ar[i],to_bn = True)
        en_bn_translated_sahih_hadiths["text_bn"][i] = text
        
    en_bn_translated_sahih_hadiths["text_en"][i] = text


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


CPU times: user 2h 20min 28s, sys: 13.7 s, total: 2h 20min 42s
Wall time: 2h 21min 14s


In [13]:

df = en_bn_translated_sahih_hadiths[en_bn_translated_sahih_hadiths['text_bn'].str.contains('এখনো পাওয়া যায়নি')]
len(df)

0

In [14]:
df = en_bn_translated_sahih_hadiths[en_bn_translated_sahih_hadiths['text_en'].str.contains('not found yet')]
len(df)

0

In [15]:
en_bn_translated_sahih_hadiths.to_csv('en_bn_translated_sahih_hadiths.csv',index = False)